# Tiktok Web Scrapping
Author: [joonath](https://github.com/joonath)

Mini-project web scrapping untuk social media Tiktok. 

### Output
Output dari mini-project ini adalah hasil scrapping dalam bentuk **excel (.xslx)** yang berisikan: 
- Link posting, 
- caption dari post, 
- tanggal posting lengkap untuk 30 post terbaru atau tanggal yang tersimplifikasi untuk post berikutnya

### Import Packages

- `webdriver` untuk handle WebDriver dan pembuatan _web scrapping instance_. WebDriver di projek ini memakai Edge (versi Chromium-based), Driver bisa didownload [disini](https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/).
- `sleep` => `sleep(n)` untuk halt eksekusi program sebanyak `n` detik
- `datetime` untuk parsing tanggal dan waktu, kalau yang didapat formatnya epoch/timestamp
- `json` untuk parsing `json` dari tag `<script>` TikTok (ada info seputar tanggal untuk 30 konten teratas)
- `pandas` untuk parsing json menuju table-like data + export (dependency tambahan: `openpyxl`)


In [1]:
from selenium import webdriver
from time import sleep
import datetime
import json
import pandas

### Setting WebDriver
- `--disable-gpu` memberitahu WebDriver untuk tidak menggunakan GPU-based processing. Argumen ini biasa dipakai kalau mau masuk ke mode `--headless`
    * `--headless`: menjalankan scrapping tanpa tampilan browser/GUI
- `--disable-extensions`: mematikan semua extension yang berjalan atau terinstall
- `--no-sandbox`: menjalankan _web scrapping instance_ tanpa melalui sandboxing (untuk menyelesaikan masalah seperti access privilege untuk scrapper, dengan tradeoff keamanan minimum)
- `--disable-notifications`: mematikan worker notification pada web yang ingin di scrap (bila ada)
- `--suppress-message-center-popups`: mematikan popup alert/dialog box

`pageLoadStrategy` [referensi](https://www.selenium.dev/selenium/docs/api/javascript/module/selenium-webdriver/lib/capabilities_exports_PageLoadStrategy.html)
- `eager`: bisa jalan saat konten di web sudah siap untuk berinteraksi
- `none`: bisa jalan saat web diakses
- `normal`: bisa jalan saat semua script dan konten selesai di load (termasuk gambar)

In [2]:
edge = webdriver.EdgeOptions()
edge.add_argument("--disable-gpu")
edge.add_argument("--disable-extensions")
edge.add_argument("--no-sandbox")
edge.add_argument("--disable-notifications")
edge.add_argument("--suppress-message-center-popups")

In [3]:
edge.capabilities["pageLoadStrategy"] = "eager"

### Setting Driver dan Web URL Starting Point 


In [4]:
url = "https://www.tiktok.com/@garyvee?lang=en"

# driver = webdriver.Edge(executable_path="C:\Program Files\edgedriver\msedgedriver.exe") #Pakai ini kalau WebDriver baru didownload, sesuaikan dengan path si exe
driver = webdriver.Edge("msedgedriver") #Pakai ini kalau WebDriver sudah di add ke environment path + restart

driver.maximize_window() #Buka web scrapping instance dengan mode Maximized
driver.get(url) #Buka URL di web scrapping instance

### Initialize Variable
Variable `gabungan` nantinya akan berisi **list of dictionary** dengan key:
- `link`, berisi URL posting
- `caption`, berisi caption posting
- `timestamp`, berisi tanggal posting

In [5]:
gabungan = []

### XPath
XPath adalah alur path untuk mendapatkan posisi suatu elemen HTML di sebuah website yang dimulai dari elemen utama (*absolute*, tag `<html>`) atau elemen terdekat (*relative*) . Cara mendapatkan XPath:
- Tekan F12 pada halaman website yang ingin dicari XPath nya. F12 menampilkan DevTools
- Gunakan element finder di bagian kiri atas DevTools (pada Microsoft Edge bisa diakses dengan shortcut `Ctrl + Shift + C`), lalu klik element yang ingin di scrap pada browser
- DevTools akan men-*highlight* tag HTML yang dipilih pada tab Elements. Klik kanan > Copy > Copy XPath

In [6]:
deret = range(10)

# Main XPath
xpath_main = "/html/body/div[2]/div[2]/div[2]/div/div[2]/div[2]/div/*[1]"
el = driver.find_element(by="xpath", value=xpath_main)

#Tiktok: Klik konten pertama/terbaru
el.click()


for i in deret:
    
    # Caption pada Tiktok dapat berupa single element apabila tidak terdapat hashtags ataupun user tags, dan sebaliknya
    # dapat berupa multiple elements apabila caption memiliki hashtags dan/atau user tags.
    # Untuk mengantisipasi hal ini, maka target elemen pada caption_xpath adalah 
    # **SEMUA ELEMEN DIDALAM div @ children 1 **
    caption_xpath = "//*[@id=\"app\"]/div[2]/div[3]/div[2]/div[2]/div[1]/*"
    
    # Karena dipastikan bisa terdapat > 1 elemen pada Caption, maka menggunakan find_elements
    capt = driver.find_elements(by="xpath", value=caption_xpath)
    
    # Initialize variabel caption_build untuk append text di setiap elemen yang ada di dalam variabel capt
    caption_build = ""
    for caption in capt:
            caption_build += caption.text + " "


    # Per Oktober 11, 2021. 
    # Timestamp versi lengkap untuk 30 posting pertama bisa didapat melalui tag <script id="SIGI_STATE">...
    # sementara sisanya bisa didapat dengan melakukan refresh terhadap halaman per-page.
    # Namun karena pertimbangan durasi scrapping, maka timestamp untuk posting berikutnya menggunakan timestamp 
    # versi simplified oleh TikTok.
    waktu_xpath = "/html/body/div[3]/script[1]"
    waktu = driver.find_element(by="xpath", value=waktu_xpath)
    
    # Setelah memastikan tag <script id="SIGI_STATE"> berisi JSON, maka dibagian ini dilakukan:
    # 1. Pengambilan seluruh isi di dalam tag tersebut
    # 2. Melakukan parsing dari yang awalnya berupa string menjadi JSON via json.loads()
    val = waktu.get_attribute("innerHTML")
    parsedJson = json.loads(val)
    
    # Mendapatkan URL di looping saat ini.
    # Sample Tiktok posting URL: 
    # https://www.tiktok.com/@garyvee/video/7152970208108383530?is_copy_url=1&is_from_webapp=v1&lang=en
    # 1. Pisah URL dengan delimiter '/' --> [https:, /, /, @garyvee, video,  7152970208108383530?is_copy_url=1&is_from_webapp=v1&lang=en]
    # 2. Ambil array terakhir dan pisah lagi dengan delimiter '?' --> [7152970208108383530, is_copy_url=1&is...]
    link = driver.current_url
    arr = link.split("/")
    terakhir = str(arr[-1]).split("?")    

    # XPath alternative timestamp untuk posting ke-31 dan seterusnya 
    alternative_crtime = "//*[@id=\"app\"]/div[2]/div[3]/div[2]/div[1]/a[2]/span[2]/span[2]"
    crtime = ""
    
    # Try Catch apabila JSON di 30 item pertama sudah habis
    try:
        # Pakai timestamp lengkap kalau ada di JSON
        crtime = datetime.datetime.fromtimestamp( int(parsedJson["ItemModule"][terakhir[0]]["createTime"]))
    except KeyError:
        # Pakai timestamp tersimplifikasi kalau tidak ada di JSON
        crtime = driver.find_element(by = "xpath", value = alternative_crtime).text
        
    # Append hasil menuju gabungan
    gabungan.append({
        "link": driver.current_url,
        "caption": caption_build,
        "timestamp": str(crtime)
    })

    # Cari tombol Next, lalu trigger klik
    next_button = "//*[@id=\"app\"]/div[2]/div[3]/div[1]/button[3]"
    nextBtn = driver.find_element(by="xpath", value=next_button)
    nextBtn.click()
    
    # Halt process selama 1 detik agar browser bisa mendapatkan waktu untuk loading
    sleep(1)

### Export ke Excel
Untuk export ke dalam file excel, digunakan `pandas` dengan melakukan parsing list of dictionary kedalam bentuk `DataFrame` 

In [7]:
dfnya = pandas.DataFrame(gabungan)

In [8]:
dfnya

,link,caption,timestamp
0,https://www.tiktok.com/@garyvee/video/71532355...,Less thinking 🤔 and more making … todays morni...,2022-10-11 19:47:39
1,https://www.tiktok.com/@garyvee/video/71530370...,Some thoughts on the changing landscape of soc...,2022-10-11 06:57:03
2,https://www.tiktok.com/@garyvee/video/71529702...,"Chase the thing that you love, people don’t be...",2022-10-11 02:37:51
3,https://www.tiktok.com/@garyvee/video/71528715...,Simple … for so many of you .. it’s time for y...,2022-10-10 20:15:11
4,https://www.tiktok.com/@garyvee/video/71526487...,Backseat seatbelts scare so few yet sharks 🦈 s...,2022-10-10 05:50:18
5,https://www.tiktok.com/@garyvee/video/71525113...,"As many of you try to build your channel, your...",2022-10-09 20:57:24
6,https://www.tiktok.com/@garyvee/video/71523469...,2 things … this video really matters to all of...,2022-10-09 10:19:04
7,https://www.tiktok.com/@garyvee/video/71522660...,Luckily we didn’t find shit 💩 this day anyway ...,2022-10-09 05:05:08
8,https://www.tiktok.com/@garyvee/video/71521855...,Tiktok a lot of you are going into this job wo...,2022-10-08 23:52:53
9,https://www.tiktok.com/@garyvee/video/71521241...,Tiktok if you see me in the “wild” make sure y...,2022-10-08 19:55:03


In [9]:
with pandas.ExcelWriter("output.xlsx") as writer:
    dfnya.to_excel(writer, sheet_name='welcome', index=False)


In [10]:
driver.quit()